## Classify name Based on Language(Country)

import neccessary modules

In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from io import open
import glob
import os

In [2]:
from __future__ import unicode_literals, print_function, division

def findFiles(path): return glob.glob(path)

print(findFiles('assets//names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )



name_char = set()
labels = []
names = []
for filename in findFiles('assets//names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    
    for name in lines:
        names.append(unicodeToAscii(name))
        labels.append(category)
        for char in name:
            if char not in name_char:
                name_char.add(unicodeToAscii(char))



[u'assets//names/Irish.txt', u'assets//names/Dutch.txt', u'assets//names/Korean.txt', u'assets//names/Italian.txt', u'assets//names/Polish.txt', u'assets//names/Scottish.txt', u'assets//names/Japanese.txt', u'assets//names/Chinese.txt', u'assets//names/English.txt', u'assets//names/Portuguese.txt', u'assets//names/Spanish.txt', u'assets//names/French.txt', u'assets//names/Arabic.txt', u'assets//names/German.txt', u'assets//names/Russian.txt', u'assets//names/Vietnamese.txt', u'assets//names/Greek.txt', u'assets//names/Czech.txt']


create a character to index dict has look up table for word encoding to vector

In [3]:
char_toIndex = dict(
                    [(char, i+1) for i, char in enumerate(name_char)])


convert word to vecotr

In [4]:
#turns each word to vector using the char_toIndex
#the num of rows is the length of the word array and the column is the 
#maximum length
def createvec(text_arr,lookup_table):
    
    arr_length = len(text_arr)
    max_text_len = max([len(text) for text in text_arr])
    
    #helps to create padding automatically
    text_vec = np.zeros((arr_length,max_text_len),dtype='int')
    
    for i, text in enumerate(text_arr):
        
        for j, char in enumerate(text):
            text_vec[i,j] = lookup_table[char]
            
            
    return text_vec

In [5]:
#Tesing how it works
text_arr = ["Dare","Dalley","Boo"]

vec = createvec(text_arr,char_toIndex)

In [6]:
vec

array([[ 9, 31, 49, 34,  0,  0],
       [ 9, 31, 43, 43, 34, 54],
       [ 7, 44, 44,  0,  0,  0]])

In [7]:
word_vec = createvec(names,char_toIndex)[:2000,:] #take a size of 20000 for easy batching

In [8]:
word_vec.shape

(2000, 19)

In [9]:
#cobvert labels to list
classes = list(set(labels))

label_dict = dict((label,i) for i, label in enumerate(classes))
data_label = []

for i in range(len(labels)):
    data_label.append(label_dict[labels[i]])

In [10]:
labels = np.array(data_label)[:2000] #convert label to array

In [11]:
labels[0:9]

array([1, 1, 1, 1, 1, 1, 1, 1, 1])

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
#split the dataset into train, test and validation
train_x, remaining_x,train_y,remaining_y = train_test_split(word_vec,labels,test_size=0.2,shuffle=True)
val_x,test_x,val_y,test_y = train_test_split(remaining_x,remaining_y,test_size=0.5,shuffle=True)

In [14]:
max_length = max([len(name) for name in names]) #the maximum length of the name

In [15]:
max_length

19

In [16]:
from torch.utils.data import TensorDataset, DataLoader

"""
convert the numpy into torch tensor
and create a tensor dataset and pass
into Dataloader to load data in batch
"""
train_data = TensorDataset(torch.from_numpy(train_x),torch.from_numpy(train_y).long())
valid_data = TensorDataset(torch.from_numpy(val_x),torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x),torch.from_numpy(test_y))

batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [17]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length

print('Sample label size: ', sample_y.size()) # batch_size


Sample input size:  torch.Size([50, 19])
Sample label size:  torch.Size([50])


In [18]:
#check if cuda is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [19]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torch.nn as nn

class ClassifyRNN(nn.Module):
    """
    The RNN model that will be used to perform classify name.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(ClassifyRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
#         self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)
        
        # embeddings and lstm_out
        embeds = self.embedding(x)
        
        
        lstm_out, hidden = self.lstm(embeds, hidden)
        

        new_out = lstm_out[:,-1,:] # take the last output
        

        out = self.dropout(new_out)
        out = self.fc(out)
        

        soft_out = F.log_softmax(out,dim=1)

        return soft_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [20]:
#assign all variables needed
vocab_size = len(name_char)+1
output_size = len(classes)
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = ClassifyRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net)

ClassifyRNN(
  (embedding): Embedding(57, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=256, out_features=18, bias=True)
)


In [21]:
lr=0.001

criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [22]:
# training params

epochs = 20 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.long())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.long())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 4/20... Step: 100... Loss: 1.861728... Val Loss: 1.716569
Epoch: 7/20... Step: 200... Loss: 1.770824... Val Loss: 1.691263
Epoch: 10/20... Step: 300... Loss: 1.741355... Val Loss: 1.693472
Epoch: 13/20... Step: 400... Loss: 1.278846... Val Loss: 1.301431
Epoch: 16/20... Step: 500... Loss: 0.719358... Val Loss: 0.736529
Epoch: 19/20... Step: 600... Loss: 0.487323... Val Loss: 0.640174


In [23]:
test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.long())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    _,pred = torch.max(output,1)
    
#     print("pred shape", pred.size())
    # compare predictions to true label
    correct_tensor = pred.eq(labels.long().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.603
Test accuracy: 0.825


In [24]:
def test_vec(text_arr,lookup_table):
    
    arr_length = len(text_arr)
#     max_text_len = max([len(text) for text in text_arr])
    
    text_vec = np.zeros((arr_length,max_length),dtype='int')
    
    for i, text in enumerate(text_arr):
        
        for j, char in enumerate(text):
            text_vec[i,j] = lookup_table[char]
            
            
    return text_vec

In [25]:
classes

[u'Portuguese',
 u'Irish',
 u'Spanish',
 u'Czech',
 u'Chinese',
 u'Vietnamese',
 u'German',
 u'Dutch',
 u'Japanese',
 u'French',
 u'Greek',
 u'Scottish',
 u'Russian',
 u'Polish',
 u'Korean',
 u'Arabic',
 u'English',
 u'Italian']

In [26]:
def predict(net,name, max_length=19):
    
    net.eval()
    
    test_ints = test_vec([name],char_toIndex)
    
    print("test_ints", test_ints.shape)
    feature_tensor = torch.from_numpy(test_ints)
    
    print("feature tensor", feature_tensor.size())
    
    batch_size = feature_tensor.size(0)
    
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    print(torch.exp(output))
    _,pred = torch.max(output,1)
    # printing output value, before rounding
#     print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    print(pred)
    print(classes[pred.item()])

In [28]:
predict(net,"Arishima")

test_ints (1, 19)
feature tensor torch.Size([1, 19])
tensor([[3.0968e-06, 2.0372e-04, 1.6461e-06, 2.3560e-06, 1.1727e-06, 2.3503e-06,
         1.0145e-06, 2.7222e-04, 9.9767e-01, 2.9914e-06, 1.6593e-06, 1.3290e-04,
         2.1683e-06, 3.7159e-04, 7.1800e-04, 1.6021e-06, 1.9931e-06, 6.1332e-04]],
       device='cuda:0', grad_fn=<ExpBackward>)
tensor([8], device='cuda:0')
Japanese
